In [1]:
%load_ext autoreload
%autoreload 2
import warnings; warnings.filterwarnings("ignore")

In [18]:
%matplotlib inline
import os
import geopandas as gp
from shapely.geometry import Polygon
#import matplotlib
#import matplotlib.pyplot as plt
import expo_nrml as nrml
import assetmaster

### define a ROI

In [19]:
lat_point_list = [-33.2,-33.0 , -33.0, -33.2, -33.2]
lon_point_list = [-71.8, -71.8,-71.4,-71.4,-71.8]

roi_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = {'init': 'epsg:4326'}
roi = gp.GeoDataFrame(index=[0], crs=crs, geometry=[roi_geom])       

### Instantiate the class and run the main function

In [9]:
#create an argument list
from argparse import Namespace
args = Namespace(assettype='res', latmax=-33.0, latmin=-33.2, 
                 lonmax=-71.4, lonmin=-71.8, schema='SARA_v1.0',
                querymode='intersects')

#instantiate the class and run the main function
model = assetmaster.Main(args)
model.run()

0

### Check the results

In [10]:
#check the results
model.query_result

,index,gc_id,name,geometry,CR+PC/LWAL,CR/LWAL/DN,CR/LWAL/_1,CR/LWAL/DU,CR/LWAL/_2,CR/LWAL/_3,...,MUR+ADO/H_,MUR+ADO/_1,MUR+STDRE/,MUR/H_1,"MUR/H_1,3",UNK,W+WLI/H_1,"W+WLI/H_1,",W+WS/H_1,"W+WS/H_1,2"
295,295,CHL.16.7.1_1,Casa Blanca,(POLYGON ((-71.63468933105463 -33.368640899658...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,297,CHL.16.7.3_1,Quilpue,(POLYGON ((-71.44433593749994 -33.082408905029...,91.5,453.6,89.3,278.2,35.2,7.8,...,51.7,712.3,87.9,4.4,1959.0,2519.0,745.5,3901.9,1159.9,1214.5
299,299,CHL.16.7.5_1,Valparaiso,(POLYGON ((-71.68458557128906 -33.102466583251...,203.4,1096.7,287.7,365.6,138.4,34.5,...,619.9,6676.5,194.0,1.0,1823.7,3648.8,1472.4,10896.5,1668.4,3327.3
301,301,CHL.16.7.7_1,Vina del Mar,(POLYGON ((-71.55708312988281 -33.014583587646...,184.8,1440.3,487.3,499.1,364.0,94.0,...,133.7,1362.6,258.2,NaN,2898.3,4931.1,1379.8,7882.7,2229.3,2384.8


### Visualize the ROI and the selected geocells

In [126]:
import folium

def plot_expo(model,taxonomies):
    
    taxlist = taxonomies.tolist()
    tmp = model.copy()
    tmp['total'] = (tmp[taxlist].sum(axis=1))
    tmp[taxlist] = tmp[taxlist].divide(tmp.total,axis='index')

    li = ['name','total']
    li.extend(taxlist)
    al = ['geocell','tot buildings'] 
    al.extend(taxlist)

    m = folium.Map([-33.08,-71.6], zoom_start=9, tiles='cartodbpositron')
    folium.GeoJson(roi).add_to(m)
    folium.GeoJson(tmp,name="test",
                   style_function=lambda x: {"weight":2, 'color':'black','fillColor':'white', 'fillOpacity':0.2},
                  highlight_function=lambda x: {'weight':3, 'color':'black'},
                   smooth_factor=2.0,
                  tooltip=folium.features.GeoJsonTooltip(fields=li,aliases=al,sticky=True,labels=True,localize=True)
                  ).add_to(m)
    #m
    return(m)

#test
plot_expo(model.query_result,model.taxonomies)